In [326]:
import sqlite3
con = sqlite3.connect("promptbase.db")
cur = con.cursor()

In [307]:
con = sqlite3.connect("promptbase.db")
cur = con.cursor()
cur.execute("CREATE TABLE prompt(rid INTEGER PRIMARY KEY, user_id, file_id, prompt_id INTEGER, instruction, problem, ans,\
            prompt_task, prompt_what, prompt_how, prompt, generated_code, prompt_source, data_source, pre_id)")
cur.execute("CREATE TABLE prompt_his(rid INTEGER PRIMARY KEY, user_id, file_id, prompt_id INTEGER, prompt, generated_code, prompt_source, data_source)")
con.commit()
con.close()

In [81]:
#res = cur.execute("DELETE FROM prompt_his")
#res = cur.execute("DELETE FROM prompt")
#con.commit()

In [322]:
res = cur.execute("SELECT DISTINCT prompt_id, prompt, prompt_source, pre_id FROM prompt WHERE file_id=1 \
                  ORDER BY prompt_id")
#con.commit()
a = res.fetchall()
a

[(0,
  'Find Unique values of each column and Print the number of unique values.\n',
  '',
  '0'),
 (0,
  "Import csv data file from the path 'credit_customers.csv' to a variable named 'credit' ",
  '',
  ''),
 (1,
  'Find unique values of each column then print the number of unique values.\n\nShow the name of each column in the loop. Only print for object type columns.\n\nShow the name of each column in the loop. Only print for object type columns.\n',
  'Metadata',
  '0')]

In [328]:
res = cur.execute("SELECT * FROM prompt_his where file_id=1")
#con.commit()
res.fetchall()

[(1,
  1,
  1,
  0,
  "Import csv data file from the path 'credit_customers.csv' to a variable named 'credit' ",
  "import pandas as pd\n\ncredit = pd.read_csv('credit_customers.csv')\n",
  '',
  ''),
 (2,
  1,
  1,
  0,
  "Import csv data file from the path 'credit_customers.csv' to a variable named 'credit' ",
  "import pandas as pd\ncredit = pd.read_csv('credit_customers.csv')\n",
  '',
  ''),
 (3,
  1,
  1,
  0,
  "Import csv data file from the path 'credit_customers.csv' to a variable named 'credit' ",
  "import pandas as pd\ncredit = pd.read_csv('credit_customers.csv')\n",
  '',
  ''),
 (4,
  1,
  1,
  0,
  "Import csv data file from the path 'credit_customers.csv' to a variable named 'credit' ",
  "import pandas as pd\n\ncredit = pd.read_csv('credit_customers.csv')\n",
  '',
  ''),
 (5,
  1,
  1,
  0,
  'Find Unique values of each column. Print the number of unique values.\n',
  "\nfor col in credit.columns:\n    print('Number of unique values in column {} is {}'.format(col, len

In [324]:
con.close()

In [218]:
from neo4j import GraphDatabase

In [219]:
uri = "neo4j://147.47.236.89:37687"
user, pw = "neo4j", "pidl2021#"

In [224]:
# insert data version from another version
def _insert_prompt(tx, prompt_id, prompt, prompt_type, prompt_source):
    query = "MERGE (n:Prompt { " \
            +"prompt_id: $prompt_id, prompt: $prompt, prompt_type: $prompt_type, prompt_source:$prompt_source}) " \
            +"RETURN n.prompt_id AS prompt_id"
    res = tx.run(query, prompt_id=prompt_id, prompt=prompt, prompt_type=prompt_type, prompt_source=prompt_source)
    print(res.data())

def _insert_prompt_edge(tx, prompt_id, pre_prompt_id):
    query = "MATCH (v1:Prompt {prompt_id: $pre_prompt_id}) " \
           +"MATCH (v2:Prompt {prompt_id: $prompt_id}) " \
           +"CREATE (v1)-[rel:NEXT_STEP]->(v2) RETURN rel "
    res = tx.run(query, pre_prompt_id=pre_prompt_id, prompt_id=prompt_id)
    print(res.data())

def insert_prompt(prompt_id, prompt, prompt_type, prompt_source, pre_prompt_id):
    driver = GraphDatabase.driver(uri, auth=(user, pw))
    with driver.session() as session:
        try:
            session.write_transaction(_insert_prompt, prompt_id, prompt, prompt_type, prompt_source)
            if pre_prompt_id:
                session.write_transaction(_insert_prompt_edge, prompt_id, int(pre_prompt_id))
        except Exception as e:
            print("[ERROR] insert node ", e)
    driver.close()

In [285]:
res = cur.execute("SELECT id, prompt, prompt_type, prompt_source, pre_id FROM prompt WHERE file_id=1 ORDER BY id")
data = res.fetchall()

In [286]:
for i in range(len(data)):
    d = list(data[i])
    d[1] = d[1].replace('Q:','').replace('\n','').strip()
    data[i] = d

In [288]:
for i in range(len(data)):
    d = data[i]
    #if i > 0 and data[i][0] == data[i-1][0]:
    #    continue
    print(d)
    #insert_prompt(d[0], d[1], d[2], d[3], d[4])

[0, "write an intitial code to import the csv data file in the path 'credit_customers.csv' to a variable named 'credit'.", 'Auto', '', '']
[0, "write an intitial code to import the csv data file in the path 'credit_customers.csv' to a variable named 'credit'.", 'Auto', '', '']
[0, "write an intitial code to import the csv data file in the path 'credit_customers.csv' to a variable named 'credit'.", 'Auto', '', '']
[1, 'print unique values of each column and the number of unique values.show the name of each column in the loop. only print for object type columns.', 'Metadata', '', '0']
[5, 'Visualizing Numerical Features by KDEplot.Please use the number of subplots equals to the number of numerical features.', 'Metadata', '', '3']
[6, 'Visualizing Categorical Features against the target feature by Countplot.Please add spacing between subplots. Please use the previous list categorical features.', 'Metadata', '', '3']
[7, 'Label Encoding for categorical feature including the target.', 'Meta

In [273]:
import nltk
from nltk import word_tokenize, pos_tag, sent_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/duclv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/duclv/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [293]:
from IPython.display import Image, display
import os
import matplotlib as mpl
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
text = word_tokenize(sent_tokenize(data[4][1])[0])
sentences = pos_tag(text)
grammar = "NP: {<DT>*<JJ>*<NN>*<IN>*<JJ>}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentences)
print(result)

no display found. Using non-interactive Agg backend
(S
  Visualizing/VBG
  (NP Numerical/JJ)
  Features/NNS
  by/IN
  KDEplot.Please/NNP
  use/VB
  (NP the/DT number/NN of/IN subplots/JJ)
  equals/NNS
  to/TO
  (NP the/DT number/NN of/IN numerical/JJ)
  features/NNS
  ./.)
